# Define Lightning based trainers

In [ ]:
#| default_exp trainers

In [ ]:
#| exports
import os
import warnings

import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch
import torch.optim as optim
import torchvision
from colorama import Fore, Style
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader

from advanced_autoencoders.dataset import MyImageDataset
from advanced_autoencoders.models import get_mae_model, get_vae_model
from advanced_autoencoders.utils import get_test_transforms, get_train_transforms

r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL
warnings.filterwarnings("ignore")

In [ ]:
#| exports
def generate_and_save_images_vq(
    dvae_model, epoch, test_sample, best_loss, cnfg, figsize=(20, 15)
):
    f, axarr = plt.subplots(1, 2, figsize=figsize)
    img1 = torchvision.utils.make_grid(test_sample.cpu()).permute(1, 2, 0).numpy()
    # axarr[0].imshow(img)

    outputs = dvae_model(test_sample)
    predictions = outputs.detach().cpu()
    img2 = torchvision.utils.make_grid(predictions).permute(1, 2, 0).numpy()

    axarr[0].imshow(img1)
    axarr[1].imshow(img2)
    axarr[0].set_title("Orignal")
    axarr[1].set_title("Reconstruction")

    if not os.path.exists(cnfg.TRAINING_RECON_IMG_DIR):
        os.makedirs(cnfg.TRAINING_RECON_IMG_DIR)
    plt.savefig(f"{cnfg.TRAINING_RECON_IMG_DIR}image_{epoch}_{best_loss}.png")

In [ ]:
#| exports
def generate_and_save_images_mae(
    model, epoch, test_sample, best_loss, cnfg, figsize=(20, 15)
):
    f, axarr = plt.subplots(1, 2, figsize=figsize)
    img1 = torchvision.utils.make_grid(test_sample.cpu()).permute(1, 2, 0).numpy()

    loss, pred, mask = model(test_sample)
    predictions = model.unpatchify(pred)
    predictions = predictions.detach().cpu()
    img2 = torchvision.utils.make_grid(predictions).permute(1, 2, 0).numpy()

    axarr[0].imshow(img1)
    axarr[1].imshow(img2)
    axarr[0].set_title("Orignal")
    axarr[1].set_title("Reconstruction")

    if not os.path.exists(cnfg.TRAINING_RECON_IMG_DIR):
        os.makedirs(cnfg.TRAINING_RECON_IMG_DIR)
    plt.savefig(f"{cnfg.TRAINING_RECON_IMG_DIR}image_{epoch}_{best_loss}.png")

In [ ]:
#| exports
class PlModelVQVAE(pl.LightningModule):
    def __init__(self, cnfg, df_train, df_val, test_sample):
        super().__init__()

        self.model = get_vae_model(cnfg)
        # self.model.load_state_dict(torch.load("./" + best_model_name + ".bin"))

        self.l_rate = cnfg.lr
        self.batch_size = cnfg.bs
        self.best_loss = 99999
        self.epoch_train_loss = None
        self.epoch_val_loss = None
        self.epoch_val_rec_loss = None
        self.validity_check_happened = False
        self.cnfg = cnfg
        self.test_sample = test_sample

        self.df_train = df_train
        self.df_val = df_val

        self.validation_step_outs = []
        self.train_step_outs = []

    def forward(
        self, inpt, return_loss=True, return_recons=False, return_encoded=False
    ):
        return self.model(
            inpt,
            return_loss=True,
            return_recons=return_recons,
            return_encoded=return_encoded,
        )

    def train_dataloader(self):
        # train
        train_dataset = MyImageDataset(
            self.df_train, augmentations=get_train_transforms(self.cnfg)
        )
        train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        return train_dl

    def val_dataloader(self):
        # valid
        valid_dataset = MyImageDataset(
            self.df_val, augmentations=get_train_transforms(self.cnfg)
        )
        valid_dl = DataLoader(valid_dataset, batch_size=self.batch_size, shuffle=False)
        return valid_dl

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)

        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.train_step_outs.append({"loss": loss})
        return loss

    def on_train_epoch_end(self):
        outputs = [k["loss"].item() for k in self.train_step_outs]
        self.epoch_train_loss = sum(outputs) / len(outputs)

        self.train_step_outs = []

        return

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            val_loss, val_recon_loss = self.forward(batch)

            self.log("val_loss", val_loss, on_epoch=True, prog_bar=True, logger=True)
            self.log(
                "val_recon_loss",
                val_recon_loss,
                on_epoch=True,
                prog_bar=True,
                logger=True,
            )

        self.validation_step_outs.append(
            {"val_loss": val_loss, "val_recon_loss": val_recon_loss}
        )
        return {"val_loss": val_loss, "val_recon_loss": val_recon_loss}

    def on_validation_epoch_end(self):
        if self.validity_check_happened:
            val_losses = [k["val_loss"].item() for k in self.validation_step_outs]
            val_recon_losses = [
                k["val_recon_loss"].item() for k in self.validation_step_outs
            ]
            self.epoch_val_loss = sum(val_losses) / len(val_losses)
            self.epoch_val_rec_loss = sum(val_recon_losses) / len(val_recon_losses)

            print(
                f"Epoch:{self.current_epoch} |Train Loss:{self.epoch_train_loss}|Valid Loss:{self.epoch_val_loss}|Rec Loss:{self.epoch_val_rec_loss}"
            )
            if self.epoch_val_rec_loss <= self.best_loss:
                print(
                    f"{g_}Rec Loss Decreased from {self.best_loss} to {self.epoch_val_rec_loss}{sr_}"
                )

                self.best_loss = self.epoch_val_rec_loss
                torch.save(
                    self.model.state_dict(),
                    f"{self.cnfg.MODELS_DIR}{self.cnfg.model_name}.bin",
                )
                with torch.no_grad():
                    generate_and_save_images_vq(
                        self.model,
                        self.current_epoch,
                        self.test_sample,
                        self.best_loss,
                        self.cnfg,
                    )
        else:
            self.validity_check_happened = True
            print("Validity check done!")

        self.validation_step_outs = []

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.l_rate)

In [ ]:
#| exports
class PlModelMAE(pl.LightningModule):
    def __init__(self, cnfg, df_train, df_val, test_sample):
        super().__init__()

        self.model = get_mae_model(cnfg)
        if cnfg.prev_model_name is not None:
            self.model.load_state_dict(torch.load("./" + cnfg.prev_model_name + ".bin"))
            print("Previous Model Loaded")

        self.best_loss = 99999
        self.epoch_train_loss = None
        self.epoch_val_loss = None
        self.epoch_val_rec_loss = None
        self.validity_check_happened = False
        self.cnfg = cnfg
        self.test_sample = test_sample
        # self.set_test_sample = False

        self.df_train = df_train
        self.df_val = df_val

        self.validation_step_outs = []
        self.train_step_outs = []

    def forward(self, inpt):
        return self.model(inpt)

    def train_dataloader(self):
        # train
        train_dataset = MyImageDataset(
            self.df_train, augmentations=get_train_transforms(self.cnfg)
        )
        train_dl = DataLoader(train_dataset, batch_size=self.cnfg.bs, shuffle=True)
        return train_dl

    def val_dataloader(self):
        # valid
        valid_dataset = MyImageDataset(
            self.df_val, augmentations=get_test_transforms(self.cnfg)
        )
        valid_dl = DataLoader(valid_dataset, batch_size=self.cnfg.bs, shuffle=False)
        return valid_dl

    def training_step(self, x, batch_idx):
        loss, pred, mask = self.forward(x)

        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        self.train_step_outs.append({"loss": loss})
        return loss

    def on_train_epoch_end(self):
        outputs = [
            k["loss"].sum().item() / self.cnfg.num_gpus for k in self.train_step_outs
        ]
        self.epoch_train_loss = sum(outputs) / len(outputs)

        self.train_step_outs = []
        return

    def validation_step(self, x, batch_idx):
        with torch.no_grad():
            # if (( not self.set_test_sample) and (self.validity_check_happened)):
            #     self.test_sample = x
            #     self.set_test_sample = True

            val_loss, pred, mask = self.forward(x)

            self.log("val_loss", val_loss, on_epoch=True, prog_bar=True, logger=True)

        self.validation_step_outs.append({"val_loss": val_loss})
        return {"val_loss": val_loss}

    def on_validation_epoch_end(self):
        if self.validity_check_happened:
            # print(validation_step_outputs)
            val_losses = [
                k["val_loss"].sum().item() / self.cnfg.num_gpus
                for k in self.validation_step_outs
            ]

            self.epoch_val_loss = sum(val_losses) / len(val_losses)

            print(
                f"Epoch:{self.current_epoch} |Train Loss:{self.epoch_train_loss}|Valid Loss:{self.epoch_val_loss}"
            )
            if self.epoch_val_loss <= self.best_loss:
                print(
                    f"{g_}Rec Loss Decreased from {self.best_loss} to {self.epoch_val_loss}{sr_}"
                )

                self.best_loss = self.epoch_val_loss
                torch.save(
                    self.model.state_dict(),
                    f"{self.cnfg.MODELS_DIR}{self.cnfg.model_name}.bin",
                )
                with torch.no_grad():
                    generate_and_save_images_mae(
                        self.model,
                        self.current_epoch,
                        self.test_sample,
                        self.best_loss,
                        self.cnfg,
                    )
        else:
            self.validity_check_happened = True
            print("Validity check done!")

        self.validation_step_outs = []

    def configure_optimizers(self):
        return optim.AdamW(
            self.parameters(),
            lr=self.cnfg.lr,
            betas=self.cnfg.betas,
            weight_decay=self.cnfg.weight_decay,
        )

In [ ]:
#| exports
def get_model_checkpoint_callback(cnfg):
    checkpoint_callback = ModelCheckpoint(
        monitor="train_loss",
        dirpath=cnfg.CHECKPOINTS_DIR + cnfg.model_name,
        filename=cnfg.model_name + "_{epoch}-{train_loss:.5f}-{val_loss:.5f}",
        save_top_k=2,
        mode="min",
    )

    return checkpoint_callback

In [ ]:
#| exports
def get_trainer_vq(cnfg, checkpoint_callback):
    trainer = Trainer(
        max_epochs=cnfg.epochs,
        fast_dev_run=False,
        callbacks=[checkpoint_callback],
        accelerator="gpu",
        devices=1,
        default_root_dir=cnfg.TRAINING_LOGS_DIR,
    )

    return trainer

In [ ]:
#| exports
def get_trainer_mae(cnfg, checkpoint_callback):
    trainer = Trainer(
        max_epochs=cnfg.epochs,
        fast_dev_run=False,
        callbacks=[checkpoint_callback],
        # strategy="dp",
        accelerator="gpu",
        # gpus=cnfg.gpus,
        devices=1,
        default_root_dir=cnfg.TRAINING_LOGS_DIR,
        # num_sanity_val_steps=0
    )

    return trainer